In [15]:
import inspect
import json
import logging
import os.path as path
from json import loads
from os import makedirs
from pathlib import Path
import csv
import os
import time
import validators
from bs4 import BeautifulSoup
from pandas import DataFrame, read_csv, concat
from requests import get, HTTPError
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import urllib.request
import re
# Create a new instance of the chrome driver
chrome_driver_path = r'C:/Users/tufan001/OneDrive - Wageningen University & Research/MIT/Food City/chromedriver.exe'

# Go to the page that we want to scrape
url = 'https://www.doordash.com/store/via-porta-new-york-2331530/?pickup=false' 
# Create a Service object
service = Service(chrome_driver_path)
# Create a WebDriver instance, passing the Service object
initial_load_time = 2
scroll_pause_time = 0.0001

In [16]:
# Create a directory to save the images
directory = 'newyork'
os.makedirs(directory, exist_ok=True)

In [17]:
# Wait until the element is present
# Create a WebDriver instance, passing the Service object
driver = webdriver.Chrome(service=service)
driver.maximize_window()
driver.get(url)
time.sleep(3)




In [20]:

opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537')]
urllib.request.install_opener(opener)
if not os.path.exists('images'):
    os.makedirs('images')
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537')]
urllib.request.install_opener(opener)

# assuming driver is already setup
name = driver.find_element(By.CSS_SELECTOR, '#__next > main > div > div.LayerManager__ChildrenContainer-sc-1k2ulq-0.gYRZjj > div > div.LayerManager__ChildrenContainer-sc-1k2ulq-0.gYRZjj > div.sc-ea0b8ed8-0.sc-93ccbd44-0.fgipDQ.fBaeps > div > div.sc-69ee586e-0.fIHkpi > div.sc-69ee586e-1.eYkXmh > div.sc-30ccff7d-0.cwDFbb > h1').text

# Clean the restaurant name to use as a directory
clean_name = re.sub(r'\W+', '_', name).lower()

# create a folder for each restaurant's menu
menu_dir = os.path.join('menus', clean_name)
os.makedirs(menu_dir, exist_ok=True)

# create a folder for each restaurant's images
img_dir = os.path.join(menu_dir, 'imgs')
os.makedirs(img_dir, exist_ok=True)


food_tags = driver.find_element(By.CSS_SELECTOR, '#__next > main > div > div.LayerManager__ChildrenContainer-sc-1k2ulq-0.gYRZjj > div > div.LayerManager__ChildrenContainer-sc-1k2ulq-0.gYRZjj > div.sc-ea0b8ed8-0.sc-93ccbd44-0.fgipDQ.fBaeps > div > div.sc-69ee586e-0.fIHkpi > div.sc-69ee586e-1.eYkXmh > div.sc-30ccff7d-0.cwDFbb > div > div > div.sc-e322ac6f-5.cAnXZy > div:nth-child(2) > span.styles__TextElement-sc-3qedjx-0.cmFuMx.sc-e322ac6f-4.ezjnHX').text

# Store the ID of the original window
original_window = driver.current_window_handle

# Check we don't have other windows open already
assert len(driver.window_handles) == 1

# Click the link which opens in a new window - google maps
driver.find_element(By.CSS_SELECTOR, ".fgipDQ.fBaeps .kSwsod > button:nth-child(2)").click()
new_window = driver.current_window_handle

# Setup wait
wait = WebDriverWait(driver, 10)
# Wait for the new window or tab
wait.until(EC.number_of_windows_to_be(2))

# Get location from google maps page
location = driver.find_element(By.CSS_SELECTOR, ".fgipDQ.fBaeps .kSwsod > button:nth-child(2)").text

#Switch back to the old tab or window restaurant page
driver.switch_to.window(original_window)

#food_tags = driver.find_element(By.CSS_SELECTOR, '#__next > main > div > div.LayerManager__ChildrenContainer-sc-1k2ulq-0.gYRZjj > div > div.LayerManager__ChildrenContainer-sc-1k2ulq-0.gYRZjj > div.sc-ea0b8ed8-0.sc-93ccbd44-0.fgipDQ.fBaeps > div > div.sc-69ee586e-0.fIHkpi > div.sc-69ee586e-1.eYkXmh > div.sc-30ccff7d-0.cwDFbb > div > div > div.sc-e322ac6f-5.cAnXZy > div:nth-child(2) > span.styles__TextElement-sc-3qedjx-0.cmFuMx.sc-e322ac6f-4.ezjnHX')

buttons = driver.find_elements(By.CSS_SELECTOR, 'div.sc-fe74847b-1.cbxTOv > div > button')

menu = {
    "name": name,
    "location": location,
    "food_tags": food_tags,
    "menu_items": []
}

# Iterate over the list of buttons and click each one
for button in buttons:
    button.click()
    time.sleep(1)  # Wait for possible page load after clicking button

    food_names = driver.find_elements(By.CSS_SELECTOR, ".sc-8370928c-0.kzJmQw h3")
    food_descriptions = driver.find_elements(By.CSS_SELECTOR, 'div.sc-8370928c-0.kzJmQw > div > span')
    prices = driver.find_elements(By.CSS_SELECTOR, '.sc-8370928c-0.kzJmQw div.InlineChildren__StyledInlineChildren-sc-6r2tfo-0.dRqGTX > div > div > span > div > span')
    imgs = driver.find_elements(By.CSS_SELECTOR, '.sc-31297892-1.hKyAGk > div > picture > img')

    # Iterate over each image, food name, description and price, and print/store them
    for food_name, food_description, price, img in zip(food_names, food_descriptions, prices, imgs):
        image_link = img.get_attribute("src")
        print(f"Food: {food_name.text}")
        print(f"Description: {food_description.text}")
        print(f"Price: {price.text}")
        print(f"Image URL: {image_link}")
        
                # This will keep only alphanumeric characters and underscores
        clean_food_name = re.sub(r'\W+', '_', food_name.text).lower()

        filename = os.path.join(img_dir, f"{clean_food_name}.jpg")  # Ensure no illegal characters in filename
        urllib.request.urlretrieve(image_link, filename)


        menu_item = {
            "food_name": food_name.text,
            "food_description": food_description.text,
            "price": price.text,
            "image_url": image_link,
        }
        menu["menu_items"].append(menu_item)

# Here you could write the menu dict to a file, or do something else with it
# Save the menu to a file inside the restaurant's directory
with open(os.path.join(menu_dir, f"{clean_name}.json"), 'w') as f:
    json.dump(menu, f)


AssertionError: 

SEARCH BY CSS


In [44]:
# Find element by CSS selector
name = driver.find_element(By.CSS_SELECTOR, '#__next > main > div > div.LayerManager__ChildrenContainer-sc-1k2ulq-0.gYRZjj > div > div.LayerManager__ChildrenContainer-sc-1k2ulq-0.gYRZjj > div.sc-ea0b8ed8-0.sc-93ccbd44-0.fgipDQ.fBaeps > div > div.sc-69ee586e-0.fIHkpi > div.sc-69ee586e-1.eYkXmh > div.sc-30ccff7d-0.cwDFbb > h1')

print(name.text)

Via Porta


In [45]:
 # Store the ID of the original window
original_window = driver.current_window_handle

In [46]:
# Check we don't have other windows open already
assert len(driver.window_handles) == 1

In [47]:
# Click the link which opens in a new window - google maps
driver.find_element(By.CSS_SELECTOR, ".fgipDQ.fBaeps .kSwsod > button:nth-child(2)").click()
new_window = driver.current_window_handle

In [48]:
# Setup wait for later
wait = WebDriverWait(driver, 10)
# Wait for the new window or tab
wait.until(EC.number_of_windows_to_be(2))

True

In [49]:
# Get location from google maps page
location = driver.find_element(By.CSS_SELECTOR, ".fgipDQ.fBaeps .kSwsod > button:nth-child(2)")
print(location.text)

522 Hudson Street
New York, NY


In [50]:
    #Switch back to the old tab or window restaurant page
driver.switch_to.window(original_window)

In [69]:
import re
import json
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537')]
urllib.request.install_opener(opener)
if not os.path.exists('images'):
    os.makedirs('images')
#name = driver.find_element(By.CSS_SELECTOR, '#__next > main > div > div.LayerManager__ChildrenContainer-sc-1k2ulq-0.gYRZjj > div > div.LayerManager__ChildrenContainer-sc-1k2ulq-0.gYRZjj > div.sc-ea0b8ed8-0.sc-93ccbd44-0.fgipDQ.fBaeps > div > div.sc-69ee586e-0.fIHkpi > div.sc-69ee586e-1.eYkXmh > div.sc-30ccff7d-0.cwDFbb > h1')
#location = driver.find_element(By.CSS_SELECTOR, ".fgipDQ.fBaeps .kSwsod > button:nth-child(2)")
#food_tags = driver.find_element(By.CSS_SELECTOR, '#__next > main > div > div.LayerManager__ChildrenContainer-sc-1k2ulq-0.gYRZjj > div > div.LayerManager__ChildrenContainer-sc-1k2ulq-0.gYRZjj > div.sc-ea0b8ed8-0.sc-93ccbd44-0.fgipDQ.fBaeps > div > div.sc-69ee586e-0.fIHkpi > div.sc-69ee586e-1.eYkXmh > div.sc-30ccff7d-0.cwDFbb > div > div > div.sc-e322ac6f-5.cAnXZy > div:nth-child(2) > span.styles__TextElement-sc-3qedjx-0.cmFuMx.sc-e322ac6f-4.ezjnHX')

buttons = driver.find_elements(By.CSS_SELECTOR, 'div.sc-fe74847b-1.cbxTOv > div > button')

menu = {
    "name": name,
    "location": location,
    "food_tags": food_tags,
    "menu_items": []
}

# Iterate over the list of buttons and click each one
for button in buttons:
    button.click()
    time.sleep(1)  # Wait for possible page load after clicking button

    food_names = driver.find_elements(By.CSS_SELECTOR, ".sc-8370928c-0.kzJmQw h3")
    food_descriptions = driver.find_elements(By.CSS_SELECTOR, 'div.sc-8370928c-0.kzJmQw > div > span')
    prices = driver.find_elements(By.CSS_SELECTOR, '.sc-8370928c-0.kzJmQw div.InlineChildren__StyledInlineChildren-sc-6r2tfo-0.dRqGTX > div > div > span > div > span')
    imgs = driver.find_elements(By.CSS_SELECTOR, '.sc-31297892-1.hKyAGk > div > picture > img')

    # Iterate over each image, food name, description and price, and print/store them
    for food_name, food_description, price, img in zip(food_names, food_descriptions, prices, imgs):
        image_link = img.get_attribute("src")
        print(f"Food: {food_name.text}")
        print(f"Description: {food_description.text}")
        print(f"Price: {price.text}")
        print(f"Image URL: {image_link}")
        
                # This will keep only alphanumeric characters and underscores
        clean_food_name = re.sub(r'\W+', '_', food_name.text)

        filename = os.path.join("images", f"{clean_food_name}.jpg")  # Ensure no illegal characters in filename
        urllib.request.urlretrieve(image_link, filename)


        menu_item = {
            "food_name": food_name.text,
            "food_description": food_description.text,
            "price": price.text,
            "image_url": image_link,
        }
        menu["menu_items"].append(menu_item)

# Here you could write the menu dict to a file, or do something else with it
with open(f"{name}.json", 'w') as f:
    json.dump(menu, f)


Food: Chicken Saltimbocca
Description: Crispy Chicken, Spicy Capicola, Arrabbiata Sauce, Provolone, Mozzarella, Pesto
Price: $14.75
Image URL: https://img.cdn4dd.com/p/fit=cover,width=1200,height=1200,format=auto,quality=80/media/photosV2/04c85384-34a0-4ef9-8c4d-d395645358c1-29d3ff1e-b439-428d-a4d9-da85d58671a6-retina-large.JPG
Food: Kale Caesar Salad
Description: Parmigiano, Toasted Walnuts, Garlic Croutons and Caesar Dressing
Price: •
Image URL: https://img.cdn4dd.com/p/fit=cover,width=1200,height=1200,format=auto,quality=80/media/photosV2/5631cf8c-e917-4b95-99de-eeaf0843c332-0a5a9058-d2d3-4baf-8028-39d7a9b5989d-retina-large.JPG
Food: Joe Vigs
Description: Mortadella, Finocchiona, Prosciutto, Asiago, Mozzarella, Peppers, Garlic Aioli
Price: 
Image URL: https://img.cdn4dd.com/p/fit=cover,width=1200,height=1200,format=auto,quality=80/media/photosV2/3771c434-da67-47fc-bc1d-2c94dc8b3bfc-a96ea271-3740-407f-9ee1-80882223ded2-retina-large.JPG
Food: Sicilian Chopped Salad
Description: Finocc

OSError: [Errno 22] Invalid argument: '<selenium.webdriver.remote.webelement.WebElement (session="affd1eac43cdff516d19eea39bc2e892", element="AAB3FE5BBAE8063F9B441A0F2627B706_element_93")>.json'

In [52]:
food_tags = driver.find_element(By.CSS_SELECTOR, '#__next > main > div > div.LayerManager__ChildrenContainer-sc-1k2ulq-0.gYRZjj > div > div.LayerManager__ChildrenContainer-sc-1k2ulq-0.gYRZjj > div.sc-ea0b8ed8-0.sc-93ccbd44-0.fgipDQ.fBaeps > div > div.sc-69ee586e-0.fIHkpi > div.sc-69ee586e-1.eYkXmh > div.sc-30ccff7d-0.cwDFbb > div > div > div.sc-e322ac6f-5.cAnXZy > div:nth-child(2) > span.styles__TextElement-sc-3qedjx-0.cmFuMx.sc-e322ac6f-4.ezjnHX')
print(food_tags.text)

Via Porta by L'Artusi


GET FOOD_NAMES

SELECT FOOD  DESCRIPTIONS

In [23]:
food_descriptions = driver.find_elements(By.CSS_SELECTOR, 'div.sc-8370928c-0.kzJmQw > div > span')
# Iterate over the list of web elements and print their text
for description in food_descriptions:
    print(description.text)

GET PRICE

In [24]:
prices = driver.find_elements(By.CSS_SELECTOR, '.sc-8370928c-0.kzJmQw div.InlineChildren__StyledInlineChildren-sc-6r2tfo-0.dRqGTX > div > div > span > div > span')
# Iterate over the list of web elements and print their text
for price in prices:
    print(price.text)

$14.75
•

$16.00
$16.25
$17.00
$14.75
•

$15.25
$5.00
$4.00
$4.00
$13.75
$6.00
$7.00
$6.00
$4.00
$4.00
$5.00
$17.00
$16.00
$9.75


GET IMG LINKS


In [25]:
import urllib.request

# Find the image element using selenium
imgs = driver.find_elements(By.CSS_SELECTOR, '.sc-31297892-1.hKyAGk > div > picture > img')

# Get the src attribute of the image element which is the url of the image
for img in imgs:
    link = img.get_attribute("src")
    print(link)


https://img.cdn4dd.com/p/fit=cover,width=1200,height=1200,format=auto,quality=90/media/photosV2/04c85384-34a0-4ef9-8c4d-d395645358c1-29d3ff1e-b439-428d-a4d9-da85d58671a6-retina-large.JPG
https://img.cdn4dd.com/p/fit=cover,width=1200,height=1200,format=auto,quality=90/media/photosV2/5631cf8c-e917-4b95-99de-eeaf0843c332-0a5a9058-d2d3-4baf-8028-39d7a9b5989d-retina-large.JPG
https://img.cdn4dd.com/p/fit=cover,width=1200,height=1200,format=auto,quality=90/media/photosV2/3771c434-da67-47fc-bc1d-2c94dc8b3bfc-a96ea271-3740-407f-9ee1-80882223ded2-retina-large.JPG
https://img.cdn4dd.com/p/fit=cover,width=1200,height=1200,format=auto,quality=90/media/photosV2/57193852-2d3d-48a5-bf66-aa54a0d5b332-retina-large.jpg
https://img.cdn4dd.com/p/fit=cover,width=1200,height=1200,format=auto,quality=90/media/photosV2/5d1c26d1-f374-489c-991d-dcc2ddd035c6-e19ba3bf-cf99-4a2b-bce5-314cc8de216c-retina-large.JPG
https://img.cdn4dd.com/p/fit=cover,width=1200,height=1200,format=auto,quality=90/media/photosV2/76b451

SAVE IMGS FROM LINKS BY 
setting a user-agent to mimic a browser request.

In [128]:
import urllib.request

opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537')]
urllib.request.install_opener(opener)

img_url = 'https://img.cdn4dd.com/p/fit=cover,width=1200,height=1200,format=auto,quality=90/media/photos/6b53872c-73a7-4b1d-97a3-79804a8bb2fe-retina-large.jpg'
urllib.request.urlretrieve(img_url, 'local_filename.jpg')


('local_filename.jpg', <http.client.HTTPMessage at 0x24bf0b81ba0>)

In [4]:
logging.basicConfig(format="%(asctime)s - [%(levelname)s]\t%(message)s",
                    datefmt='%d-%b-%y %H:%M:%S')


In [8]:
url = 'https://www.grubhub.com/search?orderMethod=pickup&locationMode=PICKUP&facetSet=umamiV6&pageSize=20&hideHateos=true&searchMetrics=true&latitude=40.72456359&longitude=-74.00174714&preciseLocation=true&geohash=dr5rsj9tf2cu&dinerLocation=POINT(-74.00174714%2040.72456359)&sorts=distance&radius=1&includeOffers=true&sortSetId=umamiv3&sponsoredSize=3&countOmittingTimes=true&tab=all'

In [6]:
# Boroughs and neighbourhoods

# Manhattan

# Financial District
# Greenwich Village and Soho
# Lower East Side and Chinatown
# Clinton and Chelsea
# Midtown
# Stuyvesant Town and Turtle Bay
# Upper West Side
# Upper East Side
# Morningside Heights and Hamilton Heights
# Central Harlem
# East Harlem
# Washington Heights and Inwood

# Bronx

# Mott Haven and Melrose
# Hunts Point and Longwood
# Morrisania and Crotona
# Highbridge and Concourse
# Fordham and University Heights
# Belmont and East Tremont
# Kingsbridge Heights and Bedford
# Riverdale and Fieldston
# Parkchester and Soundview
# Throgs Neck and Co-op City
# Morris Park and Bronxdale
# Williamsbridge and Baychester

# Brooklyn

# Greenpoint and Williamsburg
# Fort Greene and Brooklyn Heights
# Bedford Stuyvesant
# Bushwick
# East New York and Starrett City
# Park Slope and Carroll Gardens
# Sunset Park
# Crown Heights and Prospect Heights
# South Crown Heights and Lefferts Gardens
# Bay Ridge and Dyker Heights
# Bensonhurst
# Borough Park
# Coney Island
# Flatbush and Midwood
# Sheepshead Bay
# Brownsville
# East Flatbush
# Flatlands and Canarsie

# Queens

# Long Island City and Astoria
# Woodside and Sunnyside
# Jackson Heights
# Elmhurst and Corona
# Ridgewood and Maspeth
# Rego Park and Forest Hills
# Flushing and Whitestone
# Hillcrest and Fresh Meadows
# Kew Gardens and Woodhaven
# South Ozone Park and Howard Beach
# Bayside and Little Neck
# Jamaica and Hollis
# Queens Village
# Rockaway and Broad Channel

# Staten Island

# St. George and Stapleton
# South Beach and Willowbrook
# Tottenville and Great Kills


In [26]:
raw = get(url, allow_redirects=True, headers=(
    {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                        '(KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36',
    }
))

status_code = raw.status_code
reason = raw.reason

if status_code != 200:
    raise HTTPError(status_code, reason)

soup = BeautifulSoup(raw.content, 'lxml')

In [33]:
# Find all div tags
divs = soup.find_all('div')

for div in divs:
    # Get class attributes
    class_attr = div.get('class')
    if class_attr:
        print(class_attr)

['u-hidden', 'splash-screen']
['splash-screen-logo']
['splash-screen-dots']
['mainNav']
['mainNav']
['mainNavCol', 'mainNavBrand']
['errorPage-container']
['stars1']
['stars2']
['stars3']
['errorPage', 'errorPage', 'errorPage--notFound']
['errorPage-reasons']
['errorPage-image']


In [45]:
links = soup.find(text='restaurant-card')

In [49]:
soup.i

In [43]:
# Find all restaurant cards
restaurant_cards = soup.find_all('div', {'class': 'restaurant-card'})

# Loop through restaurant cards
for card in restaurant_cards:
    # Get the link to the restaurant's page
    link = card.find('a')['href']

    # Navigate to the restaurant's page
    driver.get(link)

[]
